In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import random as rn

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer



np.random.seed(42)
rn.seed(42)

In [3]:
# !pip install pennylane --no-deps

In [4]:
aids = pd.read_csv('data/AIDS_Classification.csv')

In [5]:
aids.drop(columns={'z30', 'treat','str2', 'cd420','cd820'}, inplace=True)

In [6]:
class DataFrameTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, transformer, original_columns):
        self.transformer = transformer
        self.original_columns = original_columns
    
    def fit(self, X, y=None):
        self.transformer.fit(X, y)
        return self
    
    def transform(self, X, y=None):
        X_transformed = self.transformer.transform(X)
        if not isinstance(X_transformed, pd.DataFrame):
            X_transformed = pd.DataFrame(X_transformed, columns=self.get_feature_names_out())
        
        original_data = X[self.original_columns]
        
        X_final = pd.concat([original_data.reset_index(drop=True), X_transformed.reset_index(drop=True)], axis=1)
        return X_final
    
    def get_feature_names_out(self):
        return self.transformer.get_feature_names_out()

In [7]:
cols_to_scale = ['time', 'age', 'wtkg', 'karnof', 'preanti', 'cd40', 'cd80']
cols_to_one_hot = ['trt', 'strat']
orginal_columns = [col for col in aids.columns if col not in (cols_to_scale + cols_to_one_hot)]

preprocessor = ColumnTransformer(transformers = [
    ('numerical', StandardScaler(), cols_to_scale),
    ('one_hot', OneHotEncoder(drop='first'), cols_to_one_hot)
])

pipe = Pipeline(steps = [
                ('preprocessor', DataFrameTransformer(preprocessor, orginal_columns)),
])

# joblib.dump(pipe, 'models/data_transformer.pkl')

aids_t = pipe.fit_transform(aids)

aids_t.to_csv('data/aids_t.csv')

In [8]:
X = aids_t.drop(columns=['infected'])
y = aids_t['infected']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
n_qubits = 2  # Liczba kubitów zależy od złożoności problemu
dev = qml.device("default.qubit", wires=n_qubits)


In [10]:
def quantum_layer(weights):
    # Przy założeniu jednej warstwy kwantowej
    for i in range(n_qubits):
        qml.Rot(weights[i, 0], weights[i, 1], weights[i, 2], wires=i)
    # Dodajemy splątanie między kubitami
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])


In [11]:
@qml.qnode(dev)
def quantum_neural_network(inputs, weights):
    # Kodowanie klasyczne danych do kubitów
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)

    # Warstwa kwantowa
    quantum_layer(weights)

    # Pomiar oczekiwanej wartości PauliZ na każdym kubicie
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [12]:
def cost(weights, inputs, targets):
    predictions = quantum_neural_network(inputs, weights)
    return np.mean((np.array(predictions) - np.array(targets)) ** 2)


In [13]:
# Inicjalizacja parametrów
np.random.seed(42)
weights = np.random.randn(n_qubits, 3, requires_grad=True)

# Dane wejściowe i oczekiwane wyniki
inputs = np.array([0.5, 0.1])
targets = np.array([1.0, -1.0])
# inputs = X_train
# targets = y_train

# Optymalizator
opt = AdamOptimizer(stepsize=0.1)
steps = 100  # liczba kroków optymalizacji

for i in range(steps):
    weights, cost_value = opt.step_and_cost(lambda w: cost(w, inputs, targets), weights)
    if (i + 1) % 10 == 0:
        print(f"Step {i + 1}: Cost = {cost_value:.4f}")


Step 10: Cost = 0.8918
Step 20: Cost = 0.4148
Step 30: Cost = 0.0211
Step 40: Cost = 0.0425
Step 50: Cost = 0.0195
Step 60: Cost = 0.0103
Step 70: Cost = 0.0067
Step 80: Cost = 0.0031
Step 90: Cost = 0.0012
Step 100: Cost = 0.0005


In [14]:
X_train.head()

,hemo,homo,drugs,oprior,race,gender,symptom,offtrt,numerical__time,numerical__age,numerical__wtkg,numerical__karnof,numerical__preanti,numerical__cd40,numerical__cd80,one_hot__trt_1,one_hot__trt_2,one_hot__trt_3,one_hot__strat_2,one_hot__strat_3
1808,0,1,0,1,0,1,0,0,-0.014025,-0.258212,-0.115030,0.771836,1.495736,0.603131,-0.515799,0.0,0.0,1.0,0.0,1.0
123,0,1,0,0,0,1,0,0,0.581445,0.660590,1.347104,0.771836,-0.809257,0.788713,-0.684518,0.0,0.0,1.0,0.0,0.0
1058,0,0,0,0,0,0,0,0,0.701224,-1.062163,-0.567517,0.771836,-0.809257,1.362329,-0.388738,0.0,0.0,0.0,0.0,0.0
394,0,1,0,0,1,1,0,1,0.030464,-0.947313,-0.363295,0.771836,-0.809257,1.556346,-1.076115,0.0,1.0,0.0,0.0,0.0
819,0,1,1,0,0,1,0,0,0.930514,-0.947313,0.254500,-0.923192,0.206647,0.721228,-0.353327,0.0,0.0,0.0,0.0,1.0


In [15]:
y_train.head()

1808    1
123     0
1058    0
394     0
819     0
Name: infected, dtype: int64

In [16]:
y_train

1808    1
123     0
1058    0
394     0
819     0
       ..
1638    0
1095    0
1130    1
1294    1
860     1
Name: infected, Length: 1604, dtype: int64

In [17]:
X_train = X_train.to_numpy()

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

# Przypisanie urządzenia kwantowego
# n_qubits = 6  # Liczba kubitów
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)
# dev = qml.device("lightning.qubit", wires=n_qubits)

# Warstwa kwantowa
def quantum_layer(weights):
    for i in range(n_qubits):
        qml.Rot(weights[i, 0], weights[i, 1], weights[i, 2], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])

# Model kwantowy
@qml.qnode(dev)
def quantum_neural_network(inputs, weights):
    # Kodowanie danych wejściowych
    for i in range(n_qubits):
        qml.RX(inputs[i % len(inputs)], wires=i)

    # Warstwa kwantowa
    quantum_layer(weights)

    # Oczekiwane wartości
    return qml.expval(qml.PauliZ(0))

# Funkcja kosztu
def cost(weights, X, y):
    predictions = [quantum_neural_network(X[i], weights) for i in range(len(X))]
    return np.mean((np.array(predictions) - np.array(y)) ** 2)

# Redukcja wymiaru wejściowego
def preprocess(X):
    reduced = np.tanh(np.mean(X, axis=1)).reshape(-1, 1)
    return reduced

X_train_reduced = preprocess(X_train)

# Inicjalizacja parametrów
np.random.seed(42)
weights = np.random.randn(n_qubits, 3, requires_grad=True)

# Trenowanie modelu
opt = AdamOptimizer(stepsize=0.5)
steps = 50

for step in range(steps):
    weights, cost_val = opt.step_and_cost(lambda w: cost(w, X_train_reduced, y_train), weights)
    if (step + 1) % 10 == 0:
        print(f"Step {step + 1}, Cost = {cost_val:.4f}")


Step 10, Cost = 0.2150
Step 20, Cost = 0.1818
Step 30, Cost = 0.1824
Step 40, Cost = 0.1796
Step 50, Cost = 0.1798


In [23]:
import numpy as np

# Upewnij się, że dane wejściowe są typu numerycznego
X_train = np.array(X_train, dtype=float)
y_train = np.array(y_train, dtype=int)
X_test = np.array(X_test, dtype=float)
y_test = np.array(y_test, dtype=int)

# Funkcja metrics z obliczeniem statystyk
def metrics(weights, X_train, y_train, X_test, y_test):
    # Przewidywanie wartości na zbiorze treningowym
    print('Metryki dla zbioru treningowego')
    print('-'*50)

    y_pred_prob_train = [quantum_neural_network(x, weights) for x in X_train]
    y_pred_train = (np.array(y_pred_prob_train) > 0.5).astype(int)

    # Obliczanie metryk
    accuracy_train = accuracy_score(y_train, y_pred_train)
    conf_matrix_train = confusion_matrix(y_train, y_pred_train)
    class_report_train = classification_report(y_train, y_pred_train, digits=4)

    print("Accuracy:", accuracy_train)
    print("Confusion Matrix:\n", conf_matrix_train)
    print("Classification Report:\n", class_report_train)

    print('\nMetryki dla zbioru testowego')
    print('-'*50)

    # Przewidywanie wartości na zbiorze testowym
    y_pred_prob_test = [quantum_neural_network(x, weights) for x in X_test]
    y_pred_test = (np.array(y_pred_prob_test) > 0.5).astype(int)

    # Obliczanie metryk
    accuracy_test = accuracy_score(y_test, y_pred_test)
    conf_matrix_test = confusion_matrix(y_test, y_pred_test)
    class_report_test = classification_report(y_test, y_pred_test, digits=4)

    print("Accuracy:", accuracy_test)
    print("Confusion Matrix:\n", conf_matrix_test)
    print("Classification Report:\n", class_report_test)


In [24]:
metrics(weights, X_train, y_train, X_test, y_test)

Metryki dla zbioru treningowego
--------------------------------------------------


C:\Users\Bartosz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bartosz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bartosz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.p

Accuracy: 0.7562344139650873
Confusion Matrix:
 [[1213    0]
 [ 391    0]]
Classification Report:
               precision    recall  f1-score   support

           0     0.7562    1.0000    0.8612      1213
           1     0.0000    0.0000    0.0000       391

    accuracy                         0.7562      1604
   macro avg     0.3781    0.5000    0.4306      1604
weighted avg     0.5719    0.7562    0.6513      1604


Metryki dla zbioru testowego
--------------------------------------------------
Accuracy: 0.7570093457943925
Confusion Matrix:
 [[405   0]
 [130   0]]
Classification Report:
               precision    recall  f1-score   support

           0     0.7570    1.0000    0.8617       405
           1     0.0000    0.0000    0.0000       130

    accuracy                         0.7570       535
   macro avg     0.3785    0.5000    0.4309       535
weighted avg     0.5731    0.7570    0.6523       535



C:\Users\Bartosz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bartosz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bartosz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.p

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

# Przypisanie urządzenia kwantowego
n_qubits = 6  # Liczba kubitów
dev = qml.device("default.qubit", wires=n_qubits)

# Warstwa kwantowa (zwiększona o dodatkową warstwę)
def quantum_layer(weights):
    for i in range(n_qubits):
        qml.Rot(weights[i, 0], weights[i, 1], weights[i, 2], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])
    
    # Dodanie kolejnej warstwy kwantowej
    for i in range(n_qubits):
        qml.RY(weights[i, 0], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])

# Model kwantowy
@qml.qnode(dev)
def quantum_neural_network(inputs, weights):
    # Kodowanie danych wejściowych
    for i in range(n_qubits):
        qml.RX(inputs[i % len(inputs)], wires=i)
    
    # Warstwa kwantowa
    quantum_layer(weights)

    # Oczekiwane wartości
    return qml.expval(qml.PauliZ(0))

# Funkcja kosztu
def cost(weights, X, y):
    predictions = [quantum_neural_network(X[i], weights) for i in range(len(X))]
    return np.mean((np.array(predictions) - np.array(y)) ** 2)

# Redukcja wymiaru wejściowego
def preprocess(X):
    reduced = np.tanh(np.mean(X, axis=1)).reshape(-1, 1)
    return reduced

# Przykład danych wejściowych
# Zastąpienie X_train, y_train prawdziwymi danymi
# X_train = np.random.rand(100, 6)  # Przykładowe dane (100 próbek, 6 cech)
# y_train = np.random.randint(0, 2, 100)  # Przykładowe etykiety (0 lub 1)

X_train_reduced = preprocess(X_train)

# Inicjalizacja parametrów
np.random.seed(42)
weights = np.random.randn(n_qubits, 3, requires_grad=True)

# Trenowanie modelu (zwiększenie liczby kroków)
opt = AdamOptimizer(stepsize=0.1)
steps = 200  # Zwiększenie liczby kroków

for step in range(steps):
    weights, cost_val = opt.step_and_cost(lambda w: cost(w, X_train_reduced, y_train), weights)
    if (step + 1) % 10 == 0:
        print(f"Step {step + 1}, Cost = {cost_val:.4f}")


Step 10, Cost = 0.2561
Step 20, Cost = 0.2475
Step 30, Cost = 0.2453
Step 40, Cost = 0.2450
Step 50, Cost = 0.2449
Step 60, Cost = 0.2449
Step 70, Cost = 0.2448
Step 80, Cost = 0.2448
Step 90, Cost = 0.2448
Step 100, Cost = 0.2448
Step 110, Cost = 0.2448
Step 120, Cost = 0.2448
Step 130, Cost = 0.2448
Step 140, Cost = 0.2448
Step 150, Cost = 0.2448
Step 160, Cost = 0.2448
Step 170, Cost = 0.2448
Step 180, Cost = 0.2448
Step 190, Cost = 0.2448
Step 200, Cost = 0.2448


In [26]:
metrics(weights, X_train, y_train, X_test, y_test)

Metryki dla zbioru treningowego
--------------------------------------------------
Accuracy: 0.49
Confusion Matrix:
 [[26 25]
 [26 23]]
Classification Report:
               precision    recall  f1-score   support

           0     0.5000    0.5098    0.5049        51
           1     0.4792    0.4694    0.4742        49

    accuracy                         0.4900       100
   macro avg     0.4896    0.4896    0.4895       100
weighted avg     0.4898    0.4900    0.4898       100


Metryki dla zbioru testowego
--------------------------------------------------
Accuracy: 0.297196261682243
Confusion Matrix:
 [[ 37 368]
 [  8 122]]
Classification Report:
               precision    recall  f1-score   support

           0     0.8222    0.0914    0.1644       405
           1     0.2490    0.9385    0.3935       130

    accuracy                         0.2972       535
   macro avg     0.5356    0.5149    0.2790       535
weighted avg     0.6829    0.2972    0.2201       535



In [19]:
# !pip install pennylane-lightning

In [20]:
# !pip install --upgrade pennylane pennylane-lightning
